In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
from PositionwiseFFN import PositionwiseFFN
from DotProductAttention import DotProductAttention
from MultiHeadAttention import MultiHeadAttention
from Encoder import EncoderBlock

In [5]:
# Test PFFN
d_model=64
ffn = PositionwiseFFN(dmodel)
x = torch.rand(32, 10, d_model)

assert ffn(x).shape == x.shape


In [7]:
# Test dot queries = torch.normal(0, 1, (2, 1, 2))
### Test code from d2l.ai
queries = torch.normal(0, 1, (2, 10, 2))
keys = torch.normal(0, 1, (2, 10, 2))
values = torch.normal(0, 1, (2, 10, 4))
valid_lens = torch.tensor([2, 6])

attention = DotProductAttention(dropout=0.5)
attention.eval()
assert attention(queries, keys, values).shape == (2, 10, 4)


In [9]:
# Test MHA
d_model, num_heads, inlen = 100, 5, 20
attention = MultiHeadAttention(d_model, d_model, d_model, num_heads)
batch_size = 2

X = torch.ones((batch_size, inlen, d_model))

assert attention(X, X, X, "None").shape == (batch_size, inlen, d_model)
assert attention(X, X, X, "future").shape == (batch_size, inlen, d_model)

In [10]:
# Visualize shaping of attention and masks
q = torch.rand((2,5,8))
v = torch.rand((2,5,8))
k = q

K = q.shape[1]  # Sequence length

dk = q.shape[1]**.5 
I = (q@k.transpose(1,2))

future_mask = torch.triu(torch.ones((K, K)), diagonal=1).bool()
I = I.masked_fill(future_mask, float('-inf'))
a = nn.functional.softmax(I/dk, dim=-1)@v 
#print(f"I = {I}")
#print(f"Attention = {a}")


In [12]:
# Test Encoder
dmodel=64
e = EncoderBlock(dmodel, dmodel, dmodel, h=8, dropout=0.2)
x = torch.rand(32, 10, dmodel)
assert e(x).shape == x.shape
